In [1]:
import io
import pandas as pd

In [80]:
def load_easier_example():
    example = """11111
19991
19191
19991
11111
"""
    example = example.split('\n')[:-1]
    return pd.DataFrame(list(x) for x in example).astype(int)

def load_harder_example():
    example = """5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""
    example = example.split('\n')[:-1]
    return pd.DataFrame(list(x) for x in example).astype(int)

def load_input():
    data = """4764745784
4643457176
8322628477
7617152546
6137518165
1556723176
2187861886
2553422625
4817584638
3754285662
"""
    data = data.split('\n')[:-1]
    return pd.DataFrame(list(x) for x in data).astype(int)

load_input()

,0,1,2,3,4,5,6,7,8,9
0,4,7,6,4,7,4,5,7,8,4
1,4,6,4,3,4,5,7,1,7,6
2,8,3,2,2,6,2,8,4,7,7
3,7,6,1,7,1,5,2,5,4,6
4,6,1,3,7,5,1,8,1,6,5
5,1,5,5,6,7,2,3,1,7,6
6,2,1,8,7,8,6,1,8,8,6
7,2,5,5,3,4,2,2,6,2,5
8,4,8,1,7,5,8,4,6,3,8
9,3,7,5,4,2,8,5,6,6,2


In [69]:
from itertools import product

# return all nearby positions, including diagonals
def get_nearby_positions(a, i, j):
    numrows, numcols = a.shape
    nearby_positions = []
    
    allowed_is = [i]
    allowed_js = [j]
    if i > 0:
        allowed_is.append(i-1)
    if i < numrows-1:
        allowed_is.append(i+1)
    if j > 0:
        allowed_js.append(j-1)
    if j < numcols-1:
        allowed_js.append(j+1)
        
    nearby_positions = set(product(allowed_is, allowed_js))
    nearby_positions.remove((i, j))  # remove the input position itself
    return nearby_positions

a = load_easier_example()
get_nearby_positions(a, 3, 9)

{(2, 8), (2, 9), (3, 8), (4, 8), (4, 9)}

In [75]:
# find all the 9s
def find_pos_9s(df):
    pos_9s = []
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iloc[i, j] > 9:
                pos_9s.append((i, j))

    return set(pos_9s)

# one step
def step(a):
    a = a.apply(lambda x: x+1)  # increment by 1
    # if any 9s, increment all adjacent ones by 1
    
    num_flashes = 0
    
    older_pos_to_flash = None
    flashed_already = set()
    while True:
        pos_to_flash = find_pos_9s(a)
        if not pos_to_flash:
            break
            
        # does this match the older pos_to_flash? if yes, break
        if older_pos_to_flash != None and pos_to_flash == older_pos_to_flash:
            break
        
        # increment adjacent ones
        if pos_to_flash:
            for pos_9 in pos_to_flash:
                if pos_9 in flashed_already:  # can only flash once
                    continue
                flashed_already.add(pos_9)
                num_flashes += 1
                nearby_pos = get_nearby_positions(a, pos_9[0], pos_9[1])
                for pos in nearby_pos:
                    a.iloc[pos[0], pos[1]] += 1
                    
        older_pos_to_flash = pos_to_flash.copy()

    # check for anything greater than 9 and set to 0
    a = a.mask(a > 9, 0)
    return a, num_flashes

def stringify(df):
    out = ''
    df = df.transpose()
    for x in df.to_dict().values():
        out += ''.join([str(y) for y in x.values()]) + '\n'
    return out

a = load_easier_example()
print(stringify(a))
total_num_flashes = 0
for i in range(2):
    a, num_flashes = step(a)
    total_num_flashes += num_flashes
    print(stringify(a))
    
total_num_flashes

11111
19991
19191
19991
11111

34543
40004
50005
40004
34543

45654
51115
61116
51115
45654



9

In [79]:
a = load_harder_example()
print(stringify(a))
total_num_flashes = 0
for i in range(100):
    a, num_flashes = step(a)
    total_num_flashes += num_flashes

print(stringify(a))    
total_num_flashes
    

5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

0397666866
0749766918
0053976933
0004297822
0004229892
0053222877
0532222966
9322228966
7922286866
6789998766



1656

In [81]:
a = load_input()
print(stringify(a))
total_num_flashes = 0
for i in range(100):
    a, num_flashes = step(a)
    total_num_flashes += num_flashes

print(stringify(a))    
total_num_flashes

4764745784
4643457176
8322628477
7617152546
6137518165
1556723176
2187861886
2553422625
4817584638
3754285662

7944600009
9444700000
4444709000
4444700000
4444700000
4444700000
4444700000
4444700000
4444600000
8444570007



1588

In [86]:
# part 2: figure out when they all flash simultaneously

a = load_harder_example()
print(stringify(a))
i = 0
while sum(a.sum()) != 0:
    a, num_flashes = step(a)
    i += 1

5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526



In [87]:
i

195

In [88]:
a = load_input()
print(stringify(a))
i = 0
while sum(a.sum()) != 0:
    a, num_flashes = step(a)
    i += 1

4764745784
4643457176
8322628477
7617152546
6137518165
1556723176
2187861886
2553422625
4817584638
3754285662



In [89]:
i

517